<a href="https://colab.research.google.com/github/hamagami/preprocess/blob/main/03_03_%E7%94%BB%E5%83%8F%E3%81%A8%E7%B3%BB%E5%88%97%E3%83%87%E3%83%BC%E3%82%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像と系列データの扱い
基本的な画像データや系列データの読み込みと扱い方です

In [ ]:
import pandas as pd
import urllib.request 
import numpy as np
from PIL import Image #画像を扱うモジュールはいくつかありますが最も基本的なpython image library(PIL)を例にあげます
import matplotlib.pyplot as plt
import urllib.request 
%matplotlib inline

## 画像データの読み込み

In [ ]:
url = "https://dl.dropbox.com/s/9zmhzdrvse8h7n6/yokohama01.jpg" #画像データ
urllib.request.urlretrieve(url, 'yokohama.jpg') # titanicという名前で google colabのドライブに保存します


In [ ]:
img= Image.open("yokohama.jpg")  # PIL(Python Image Library)で読みこみます

In [ ]:
#画像データをnumpy形式にしてマトリクスデータにします，サイズを確認します　(x, y, depth )になります
img_array = np.array(img)
print(img_array.shape)

### 画像を出力してみます

In [ ]:
plt.imshow(img_array)
plt.title("Yokohama")
plt.show()

## データの中身を直接をみてみます
numpy オブジェクトをそのまま評価してもprint文にいれてもいいです

In [ ]:
img_array

In [ ]:
print(img_array) # hight x width ×　depth のテンソルになっています

## １チャネルだけを取り出して行列にして表示させてみる

In [ ]:
R = img_array[:, :, 0] # R  0ならばRチャネル
G = img_array[:, :, 1] # G  1ならばGチャネル
B = img_array[:, :, 2] # B　2ならばBチャネル　
# ただし，PILでない別の画像処理モジュールだと別の並びになっていることがあるので注意です。

In [ ]:
plt.imshow(R,cmap="gray") # Rの濃淡をグレーで表現
plt.title("R channel")
plt.show()
plt.imshow(G,cmap="gray") # Gの濃淡をグレーで表現
plt.title("G channel")
plt.show()
plt.imshow(B,cmap="gray") # Bの濃淡をグレーで表現
plt.title("B channel")
plt.show()

## RGB合わせて正しくグレースケールにする




In [ ]:
img_gray = 0.299 * img_array[:, :, 0] + 0.587 * img_array[:, :, 1] + 0.114 * img_array[:, :, 2] #視覚的に正しいグレイスケール化(1:1:1ではないです)

In [ ]:
plt.imshow(img_gray,cmap="gray")
plt.title("gray scale")
plt.show()

グレースケールにしたデータの中身を直接みてみます。マトリクスになっています

In [ ]:
print(img_gray.shape)# 中身は　hight x width の行列
print(img_gray)

##  2次元データを1次元に変えて(flatten) その結果をヒストグラムでみる　（輝度分布)

In [ ]:
pixdata= img_gray.flatten()
print(pixdata.shape)
pixdata #1次元になっているかどうか表示してみます

In [ ]:
plt.hist(pixdata,bins=50) #binの数を50
plt.title("Luminance dist")
plt.show()



---



---



## 系列データの読み込み
コロナの新規陽性者数のデータを厚労省のページからもってきます

In [ ]:
url = "https://covid19.mhlw.go.jp/public/opendata/newly_confirmed_cases_daily.csv" #新規陽性者数の推移（日別）
urllib.request.urlretrieve(url, 'daily.csv') # daily.csvという名前で google colabのドライブに保存します

In [ ]:
df= pd.read_csv("daily.csv")  # PIL(Python Image Library)で読みこみます
df #データの様子をみてみます

## データの型を少し変えてみる

In [ ]:
print(df.dtypes) # 単に読みこんだだけだと　date が Object型になっています　このままでも問題ありませんが，あとで時間軸の操作も行いたいので日付型に変換します
df["Date"] = pd.to_datetime(df["Date"])
print(df.dtypes)

もう１度表示して型が変わっていることを確認

In [ ]:
df

# 都道府県ごとにデータを取り出してみる

In [ ]:
tokyo_df = df[df["Prefecture"] == "Tokyo"] # Tokyo だけのデータ
osaka_df = df[df["Prefecture"] == "Osaka"] # Oaka だけのデータ
kanagawa_df= df[df["Prefecture"] == "Kanagawa"] # Kanagawa だけのデータ

東京のデータ

In [ ]:
tokyo_df

## グラフにしてみる

In [ ]:
plt.plot(tokyo_df["Newly confirmed cases"].values, label="Tokyo")
plt.plot(osaka_df["Newly confirmed cases"].values, label="Osaka")
plt.plot(kanagawa_df["Newly confirmed cases"].values, label="Kanagawa")
plt.legend()
plt.show()

## フレーム分析
時系列データのうち，一定の時間幅（フレーム）でその時間内の特徴を捉える方法をフレーム分析といいます。一定の幅のデータを時間をずらしながら分析します
一番単純なフレーム分析は移動平均です。rollingを使うと便利です

In [ ]:
#ここでは東京のデータをつかって1週間(7日)の平均値が求まるように平均をとります。
tokyo_mean = pd.DataFrame({"Date":tokyo_df["Date"],"Average 1w cases":tokyo_df['Newly confirmed cases'].rolling(window=7).mean()})
tokyo_mean


In [ ]:
plt.plot(tokyo_df["Newly confirmed cases"].values, label="Tokyo")
plt.plot(tokyo_mean["Average 1w cases"].values, label="moving ave")
plt.legend()